In [ ]:
# 📦 Install missing libraries if needed
!pip install transformers peft bitsandbytes --quiet

# ========================
# 🔥 Setup
# ========================
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# 📂 Paths
base_model = "mistralai/Mistral-7B-v0.1"
lora_weights_path = "outputs/lora_weights"

# 🧪 Test prompts
test_prompts = [
    "My baby keeps crying when I put her down. Any advice?",
    "I yelled at my kid today. Feeling horrible.",
    "How do you keep your marriage strong after having kids?",
    "My 3 y/o won't eat anything but nuggets. Help?",
    "I just need a break. Is it bad to want one?"
]

# ========================
# 🚀 Load Model + LoRA
# ========================
print("🔄 Loading model...")

bnb_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto")

model = PeftModel.from_pretrained(model, lora_weights_path)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model.eval()

print("✅ Model ready!")

# ========================
# 🧪 Batch Inference
# ========================
def generate_reply(prompt_text):
    full_prompt = f"Post Reddit : {prompt_text}\n\nDad's reply:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = decoded.split("Dad's reply:")[-1].strip()
    return reply

# ========================
# 🎯 Test prompts
# ========================
print("\n📨 Running tests...\n")

for idx, prompt_text in enumerate(test_prompts, 1):
    reply = generate_reply(prompt_text)
    print(f"🟰 Prompt {idx}: {prompt_text}")
    print(f"🤖 DadAI: {reply}")
    print("-" * 60)